# Homework 3: vector search

In this notebook, we will explore vector search using embeddings with and without Elasticsearch. The process involves obtaining embeddings for a given query and a set of documents, performing similarity searches, and evaluating the effectiveness of the search engine using metrics such as hit-rate. We will also compare the performance of our custom vector search engine against Elasticsearch. This exercise aims to provide practical insights into implementing and evaluating vector search techniques.

## Sentence Transformers

In this step, we import the `SentenceTransformer` class from the `sentence_transformers` library. We then load a pre-trained model named `multi-qa-distilbert-cos-v1` which is specifically designed for question-answering tasks. This model will be used to generate embeddings for our queries and documents, allowing us to perform vector search based on the similarity of these embeddings.

Check the [Senterce Transformers' model library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html) if you're interested in checking out other models that are available. 

This is the information of `multi-qa-distilbert-cos-v1`:

| **Attribute**             | **Details**                                                                                                                                                                    |
|---------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Description**           | This model was tuned for semantic search: Given a query/question, it can find relevant passages. It was trained on a large and diverse set of (question, answer) pairs.        |
| **Base Model**            | distilbert-base                                                                                                                                                                |
| **Max Sequence Length**   | 512                                                                                                                                                                            |
| **Dimensions**            | 768                                                                                                                                                                            |
| **Normalized Embeddings** | true                                                                                                                                                                           |
| **Suitable Score Functions** | dot-product (util.dot_score), cosine-similarity (util.cos_sim), euclidean distance                                                                                          |
| **Size**                  | 250 MB                                                                                                                                                                         |
| **Pooling**               | Mean Pooling                                                                                                                                                                   |
| **Training Data**         | 215M (question, answer) pairs from diverse sources.                                                                                                                            |
| **Model Card**            | [https://huggingface.co/sentence-transformers/multi-qa-distilbert-cos-v1](https://huggingface.co/sentence-transformers/multi-qa-distilbert-cos-v1)                             |


In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [2]:
query = "I just discovered the course. Can I still join it?"

query_embedding = embedding_model.encode(query)
print(f"This embedding has a dimensionality of {len(query_embedding)}. The first values of this vector are {query_embedding[:5]}")

This embedding has a dimensionality of 768. The first values of this vector are [ 0.07822261 -0.04013115  0.03861361 -0.00017896  0.08923467]


## Data loading 

Now we will download the documents we will use for this homework. We also save a copy of these documents for reproductibility.

In [3]:
import requests
import json

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'

docs_response = requests.get(docs_url)
documents = docs_response.json()

# Save the fetched documents locally as documents.json
with open('documents-with-ids.json', 'w') as f:
    json.dump(documents, f)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
# We will use only a subset of the questions: the questions belonging to "machine-learning-zoomcamp".
ml_documents = [document for document in documents if document['course'] == "machine-learning-zoomcamp"]

## Compute the embeddings

In this step, we generate embeddings for each document using the pre-trained model. These vectors are stored in a list, which is then converted to a numpy array for efficient numerical operations.

In [5]:
import numpy as np
from tqdm import tqdm

embeddings = []
for document in tqdm(ml_documents):
    qa_text = f"{document['question']} {document['text']}"
    embeddings.append(embedding_model.encode(qa_text))
X = np.array(embeddings)
X.shape

100%|██████████| 375/375 [00:32<00:00, 11.65it/s]


(375, 768)

In this step, we compute the dot product of the first embedding vector with itself to show that the vectors returned from the embedding model are already normalized:

In [6]:
dot_product = X[0].dot(X[0])
dot_product

1.0

Now we compute the similarity between the query embedding and all document embeddings using the dot product. We then identify the document with the highest similarity score by iterating through the computed similarities and finding the maximum value.

In [7]:
similarities = X.dot(query_embedding)
highest_similarity = max(enumerate(similarities), key=lambda x: x[1])
print(f"Document #{highest_similarity[0] + 1} has the highest similarity with a dot product of {highest_similarity[1]}")
ml_documents[highest_similarity[0]]

Document #15 has the highest similarity with a dot product of 0.6506574153900146


{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'id': 'ee58a693'}

## Creating a Search Engine

Now we define a `VectorSearchEngine` class to facilitate efficient document retrieval based on vector similarity. The class takes a list of documents and their corresponding embeddings as inputs during initialization. The `searc`h` method computes similarity scores between a query embedding and all document embeddings using the dot product.

To efficiently find the top `num_results` scores, we use `np.argpartition` which is more efficient than `np.argsort` when we only need the top results sorted. This improves the search performance by avoiding a full sort of all scores. In the appendix, we discuss the differences and efficiencies of `np.argpartition` and `np.argsort` functions.

In [8]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        partition_idx = np.argpartition(scores, -num_results)[-num_results:]
        idx = partition_idx[np.argsort(-scores[partition_idx])]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
search_engine.search(query_embedding, num_results=2)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Ground truth data

In this section, we load the ground truth data for the evaluation of our search engine.

In [9]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)

df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']

ground_truth = df_ground_truth.to_dict(orient='records')

df_ground_truth.to_json('ground_truth.json', orient='records', lines=True)

The data contains relevant questions and the corresponding document IDs that answer these questions. Our strategy is to generate embeddings for all questions and evaluate the search engine's performance by checking if the top 5 retrieved results contain the ground truth document ID.

In [10]:
ground_truth[:3]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

## Evaluation Metrics

The ground truth data will be used to calculate the evaluation metrics. Here we define the evaluation functions and describe their purpose:

**Hit Rate (HR) or Recall at k:**

Measures the proportion of queries for which at least one relevant document is retrieved in the top k results.

$$
HR@k = \frac{\text{Number of queries with at least one relevant document in top k}}{|Q|}
$$

**Mean Reciprocal Rank (MRR):**

Evaluates the rank position of the first relevant document.

$$
MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \left( \frac{1}{\text{rank}_i} \right)
$$

where $ |Q| $ is the total number of queries and $ \text{rank}_i $ is the rank position of the first relevant document for the i-th query.

`evaluate` performs the embedding of the query question, uses the search function to retrieve the top results, and checks if the ground truth document is among the retrieved results. It creates a list of booleans indicating matches for each document and calculates the HR and MRR metrics based on these matches.

In [11]:
def hit_rate(matches_list):
    cnt = 0

    for line in matches_list:
        if True in line:
            cnt = cnt + 1

    return cnt / len(matches_list)

def mrr(matches_list):
    total_score = 0.0

    for line in matches_list:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(matches_list)

def evaluate(ground_truth, search_function):
    matches_list = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        matches = [d['id'] == doc_id for d in results]
        matches_list.append(matches)

    return {
        'hit_rate': hit_rate(matches_list),
        'mrr': mrr(matches_list),
    }

The `search` function takes an entry from the ground truth as input, encodes the question using the embedding model, and then uses the search engine to retrieve the top 5 results based on the vector similarity. We then use the evaluate function to assess the performance of our search engine using the ground truth data.

In [12]:
def search(q):
    question = q['question']
    v_q = embedding_model.encode(f"{question}")

    return search_engine.search(v_q, num_results=5)

evaluate(ground_truth, search_function=search)

100%|██████████| 1830/1830 [00:41<00:00, 43.81it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

## Searching using Elasticsearch

Now we will perform the search with Elasticsearch. Make sure you have a container running Elasticsearch before proceeding. For a brief intro to Elasticsearch and an example of running such a container is found [here](https://github.com/Fustincho/datatalks-llm-zoomcamp/blob/main/01-intro/elasticsearch.ipynb).

We will index the `ml_documents`. For this we must define the mappings for every key found in our documents. We will also add a new key, `question_text_vector` in which we will add the embedding vector before indexing it into Elasticsearch.

In [13]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = 'ml_questions'

# Delete the index if it already exists
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml_questions'})

In [14]:
for i in range(len(ml_documents)):
    ml_documents[i]["question_text_vector"] = X[i]
    
for doc in tqdm(ml_documents):
    es.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:01<00:00, 286.55it/s]


In this section, we define a function `elastic_search_knn` to perform k-Nearest Neighbors (k-NN) search using Elasticsearch. The function takes a field name and a query vector as inputs. It constructs a search query with the specified field and query vector, specifying the number of nearest neighbors (k) to return and the number of candidates to consider (`num_candidates`). The search query is executed against the Elasticsearch index, and the resulting documents are extracted and returned as a list.

In [15]:
def elastic_search_knn(field, query_vector):

    search_query = {
        "knn": {
            "field": field,
            "query_vector": query_vector,
            "k": 5,
            "num_candidates": 10000,
        },
        "_source": [
            "text",
            "section",
            "question",
            "course",
            "id",
        ],  # Specifies what elements from the source document should be returned
    }

    es_results = es.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [16]:
elastic_search_knn("question_text_vector", query_embedding)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

Here, the search function `search_es` uses Elasticsearch for retrieving the top results. The function takes a query as input, encodes the question using the embedding model, and performs a k-NN search using the `elastic_search_knn` function.

We then evaluate the search engine's performance using the ground truth data and the evaluate function. 

In [17]:
def search_es(q):
    question = q['question']
    v_q = embedding_model.encode(f"{question}")

    return elastic_search_knn("question_text_vector", v_q)

evaluate(ground_truth=ground_truth, search_function=search_es)

100%|██████████| 1830/1830 [00:29<00:00, 61.07it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

# Appendix 1: Finding Indices of Highest Values for Vector Similarities in Search Engine

When implementing a search engine that ranks results based on vector similarities, it is often necessary to find the indices of the highest similarity values. Depending on the number of top results you need, different functions in `numpy` can be used efficiently.

## 1. Using `numpy.argmax`
- **Purpose**: Find the index of the single highest similarity value in an array.
- **Time Complexity**: `O(n)`
- **Best Use Case**: When you need to find the index of the single highest similarity value.
- **Example**:
  ```python
  import numpy as np

  similarities = np.array([0.3, 0.1, 0.2, 0.5, 0.4])
  max_index = np.argmax(similarities)
  ```

## 2. Using `numpy.argsort`
- **Purpose**: Find the indices of the sorted array of similarity values.
- **Time Complexity**: `O(n log n)`
- **Best Use Case**: When you need a fully sorted array or a sorted order of similarity values, including their indices.
- **Example**:
  ```python
  import numpy as np

  similarities = np.array([0.3, 0.1, 0.2, 0.5, 0.4])
  sorted_indices = np.argsort(similarities)
  top_k_indices = sorted_indices[-5:]  # For top 5 similarity values
  ```

## 3. Using `numpy.argpartition`
- **Purpose**: Find the indices of the top k highest similarity values.
- **Time Complexity**: `O(n)` for partitioning + `O(k log k)` for sorting the top k elements.
- **Best Use Case**: When you need the top k highest similarity values efficiently, especially for large arrays and small k.
- **Example**:
  ```python
  import numpy as np

  similarities = np.array([0.3, 0.1, 0.2, 0.5, 0.4, 0.7, 0.6, 0.8, 0.9, 0.0])
  k = 5
  partitioned_indices = np.argpartition(similarities, -k)[-k:]
  top_k_indices = partitioned_indices[np.argsort(-similarities[partitioned_indices])]
  ```
np.argpartition rearranges the indices of the array such that all values before the value at the k-th position are less than or equal to it, and all values after are greater than or equal to it. This operation ensures that the k-th largest element is in its final sorted position, but the order of the elements in the two partitions (before and after the k-th element) is not guaranteed to be sorted.

In the example, `np.argpartition(similarities, -k)` partitions the array similarities such that the indices of the top k highest values are placed in the last k positions of the array.
`partitioned_indices = np.argpartition(similarities, -k)[-k:]` retrieves these indices.

`np.argsort(-similarities[partitioned_indices])` sorts these top k indices based on the actual similarity values in descending order.